In [1]:
!pip install boto3
!pip install sentencepiece
!pip install transformers
!pip install torch

     |████████████████████████████████| 133kB 30.4MB/s 
     |████████████████████████████████| 7.7MB 28.6MB/s 
     |████████████████████████████████| 81kB 9.4MB/s 
     |████████████████████████████████| 143kB 44.4MB/s 
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.6 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
     |████████████████████████████████| 1.2MB 25.4MB/s 
     |████████████████████████████████| 2.5MB 34.3MB/s 
     |████████████████████████████████| 3.3MB 38.0MB/s 
     |████████████████████████████████| 901kB 37.8MB/s 
     |████████████████████████████████| 133kB 51.5MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing inst

In [2]:
import torch

In [22]:
import truecase

In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

In [5]:
with open('rootkey.csv', 'r') as f:   #Get AWSKeys from rootkey.csv file
    content = f.readlines()

keys = {}
for line in content:
    pair = line.strip().split('=')
    keys.update({pair[0] : pair[1]})

AWS_ACCESS_KEY_ID = keys['AWSAccessKeyId']
AWS_SECRET_KEY = keys['AWSSecretKey']

In [6]:
import boto3  #boto3 is a libary to interact with aws

BUCKET_NAME ='awstranscriptmk'
FILE_NAME ='converted.wav'
JOB_NAME ='mk_test' # can be anything

# Instantiate a client to the AWS transcribe service
transcribe = boto3.client(
    'transcribe',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name='us-west-2')

In [7]:
def start_transcribe_job(transcribe,job_name,bucket,file):
    """Starts an AWS Transcribe job
   
    Parameters
    ----------
    transcribe : AWS `transcribe` service client instance
    job_name : str, name of the AWS job
    bucket : str, AWS S3 bucket name
    file : str, name of the mp3 file to transcribe
    
    Returns
    -------
    True: if job started successfully
    
    """
    # this is the location of audio file
    file_uri = f'https://s3.amazonaws.com/{bucket}/{file}'
    
    # try launching the job and return any error messages
    try:
        transcribe.start_transcription_job(
            TranscriptionJobName=JOB_NAME,
            Media={'MediaFileUri': file_uri},
            MediaFormat='wav',
            LanguageCode='en-US')
        return True
    except Exception as e:
        return e

In [8]:
def get_transcription_text(transcribe,job_name):
    """Returns transcription text for the AWS Transcribe job
    
    Parameters
    ----------
    transcribe : AWS `transcribe` service client instance
    job_name : transcribe service `job` name
    
    Returns
    -------
    Current job status if job is still in progress
    Transcription text if job is Completed
    """
    import urllib.request
    import json
    import time
    
    # let's obtain the job instance
    job = transcribe.get_transcription_job(TranscriptionJobName=job_name)
    
    # and it's status
    status = job['TranscriptionJob']['TranscriptionJobStatus']
    
    # check the status every 5 seconds and 
    # return the transcribed text if the job is finished
    # otherwise return None if job failed
    while True:
        if status == 'COMPLETED':
            print(f"Job {job_name} completed")
            with urllib.request.urlopen(job['TranscriptionJob']['Transcript']['TranscriptFileUri']) as r:
                data = json.loads(r.read())
            return data['results']['transcripts'][0]['transcript']
        elif status == 'FAILED':
            print(f"Job {job_name} failed")
            return None
        else:
            print(f"Status of job {job_name}: {status}")
            time.sleep(5)

In [11]:

job_status = start_transcribe_job(transcribe,JOB_NAME,BUCKET_NAME,FILE_NAME)

# if job launched successfully `job_status` will be True
if job_status: # and we can start requesting the results from the service
    text = get_transcription_text(transcribe,JOB_NAME)
    print(f'The transcribed text for {FILE_NAME} file:')
    print(text)
    with open('result.txt',mode='w') as file :
     file.write(text)
  

else: # or print the error code if somethign went wrong
    print(f'Job {JOB_NAME} failed with the error: {job_status}')

Job mk_test completed
The transcribed text for converted.wav file:
Hello. My name is Kirsten Freeman. I'm originally from Denver colorado but was raised right here in Knoxville Tennessee. Today I'll be sharing a little bit more about my background, my interest in the holidays and my plans for the future. As I mentioned before, I was raised in Knoxville. My elementary middle and high school are all within a 15 mile radius of the University of Tennessee. I've been playing sports since I was six years old, which was a crucial part of my childhood and helped me become who I am today. I'm also a very competitive person. This helped bring a sense of selflessness at a very young age because I was willing to sacrifice anything to win games and reach my desired and result this sense of selflessness would help me in the future. When I became more interested in service clubs, my interest in entering the service groups only increases I would throughout school, by my senior in high school. This int

In [10]:
model = T5ForConditionalGeneration.from_pretrained('t5-large')  #t5-base  is a pretrained model.
tokenizer = T5Tokenizer.from_pretrained('t5-large')  


In [12]:
device = torch.device('cpu')   #use the  cpu for training 

In [14]:
f=open('result.txt','r')

In [15]:
text=f.read()

In [16]:
preprocess_text = text.strip().replace("\n","")  #remove newlines and append space
t5_prepared_Text = "summarize: "+preprocess_text 
print ("original text preprocessed: \n", preprocess_text)

tokenized_text = tokenizer.encode(t5_prepared_Text,return_tensors="pt").to(device)

original text preprocessed: 
 Hello. My name is Kirsten Freeman. I'm originally from Denver colorado but was raised right here in Knoxville Tennessee. Today I'll be sharing a little bit more about my background, my interest in the holidays and my plans for the future. As I mentioned before, I was raised in Knoxville. My elementary middle and high school are all within a 15 mile radius of the University of Tennessee. I've been playing sports since I was six years old, which was a crucial part of my childhood and helped me become who I am today. I'm also a very competitive person. This helped bring a sense of selflessness at a very young age because I was willing to sacrifice anything to win games and reach my desired and result this sense of selflessness would help me in the future. When I became more interested in service clubs, my interest in entering the service groups only increases I would throughout school, by my senior in high school. This interest had turned into passion. I was 

In [17]:
# crate summary 
summary_ids = model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=60,
                                    max_length=150,
                                    early_stopping=False)

output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print ("\n\nSummarized text: \n",output)

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)




Summarized text: 
 kirsten freeman was raised in Knoxville, tennessee. she's been involved with five service clubs within her school and in the community. her end goal, however, is to become a high school principal in this position, she would love to remain heavily involved in student life.


In [18]:
!pip install truecase

     |████████████████████████████████| 28.4MB 102kB/s 


In [23]:
truecase.get_true_case(output)

"Kirsten Freeman was raised in Knoxville, Tennessee. She's been involved with five service clubs within her school and in the community. Her end goal, however, is to become a high school principal in this position, she would love to remain heavily involved in student life."